In [1]:
import csv
import cobra.mit.access
import cobra.model.fv
import cobra.mit.request
import cobra.mit.session
import cobra.model.fvns
import cobra.model.pol
import cobra.model.phys
import cobra.model.vns
import cobra.model.bgp
import cobra.model.vz
import cobra.model.leak
import cobra.model.pol
import cobra.model.infra
import cobra.model.vns
from cobra.internal.codec.xmlcodec import toXMLStr
import cobra.model.infra
from cobra.mit.request import ClassQuery
from cobra.mit.session import LoginSession
from cobra.mit.access import MoDirectory
import json
import re
from re import match
from cobra.internal.codec.jsoncodec import toJSONStr
import getpass
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()

In [ ]:

#import json
#from cobra.internal.codec.jsoncodec import toJSONStr
#cq = cobra.mit.request.ClassQuery('fvTenant')
#cq = cobra.mit.request.ClassQuery('uni/infra')
# set subtree to full
#cq.subtree = 'full'
# set property filter to tenant name
#cq.propFilter = 'eq(fvTenant.name, "Daniel")'

# submit query
#tnt = md.query(cq)
#for mo in tnt:
x=(json.loads(toJSONStr(mo)))

In [2]:
def parseClass(classDn):
    matchClass=re.findall("([^/]*)$", classDn)
    return(matchClass[0])

In [30]:
def classQueryFunc(classDn):
    matchClass=parseClass(classDn)
    classQuery = ClassQuery("/"+matchClass)
    classQuery.subtree = 'full'
    tenantBdMoList = md.query(classQuery)
    temp_list=[]
    for tenantBd in tenantBdMoList:
        temp_list.append(tenantBd.dn)
    return(temp_list)


In [31]:
def querydnjson_funct(classDn):
    classQuery = ClassQuery(classDn)
    classQuery.subtree = 'full'
    MoList = md.query(classQuery)
    for mo in MoList:
        return(json.loads(toJSONStr(mo)))

In [32]:
def createVlanPool(md,vlanname,vlanrange):

    res = re.split('-', vlanrange)
    fromvlan=('vlan-'+res[0])
    tovlan=('vlan-'+res[1])
    # the top level object on which operations will be made
    polUni = cobra.model.pol.Uni('')
    infraInfra = cobra.model.infra.Infra(polUni)

    # build the request using cobra syntax
    fvnsVlanInstP = cobra.model.fvns.VlanInstP(infraInfra, allocMode='static', annotation='', descr='', name=vlanname, nameAlias='', ownerKey='', ownerTag='', userdom=':all:')
    fvnsEncapBlk = cobra.model.fvns.EncapBlk(fvnsVlanInstP, allocMode='inherit', annotation='', descr='', from_=fromvlan, name='', nameAlias='', role='external', to=tovlan, userdom=':all:')


    # commit the generated code to APIC
    print(toXMLStr(infraInfra))
    c = cobra.mit.request.ConfigRequest()
    c.addMo(infraInfra)
    md.commit(c)

In [33]:


def createPhysDom(md,vlanpool,physdomname):
    
    vlandn = re.sub('\/fvnsVlanInstP$', '', vlanpool)
    print(vlandn)
    # the top level object on which operations will be made
    polUni = cobra.model.pol.Uni('')
    # build the request using cobra syntax
    physDomP = cobra.model.phys.DomP(polUni, annotation='', name=physdomname, nameAlias='', ownerKey='', ownerTag='', userdom=':all:')
    infraRsVlanNs = cobra.model.infra.RsVlanNs(physDomP, annotation='', tDn=vlandn, userdom=':all:')

    # commit the generated code to APIC
    print(toXMLStr(polUni))
    c = cobra.mit.request.ConfigRequest()
    c.addMo(polUni)
    md.commit(c)

In [34]:
def attachdomaep(md,aep,physdomname):
    # the top level object on which operations will be made
    polUni = cobra.model.pol.Uni('')
    infraInfra = cobra.model.infra.Infra(polUni)
    infraAttEntityP = cobra.model.infra.AttEntityP(infraInfra, aep)

    # build the request using cobra syntax
    infraRsDomP = cobra.model.infra.RsDomP(infraAttEntityP, annotation='', forceResolve='yes', rType='mo', tCl='physDomP', tDn='uni/phys-'+physdomname, tType='mo', userdom=':all:')


    # commit the generated code to APIC
    print(toXMLStr(infraAttEntityP))
    c = cobra.mit.request.ConfigRequest()
    c.addMo(infraAttEntityP)
    md.commit(c)

In [35]:
def createTenant(md,tenantname):



    # the top level object on which operations will be made
    polUni = cobra.model.pol.Uni('')

    # build the request using cobra syntax
    fvTenant = cobra.model.fv.Tenant(polUni, annotation='', descr='', name=tenantname, nameAlias='', ownerKey='', ownerTag='', userdom=':all:')
    vzBrCP = cobra.model.vz.BrCP(fvTenant, annotation='', descr='', intent='install', name='any', nameAlias='', ownerKey='', ownerTag='', prio='unspecified', targetDscp='unspecified', userdom=':all:')
    vnsSvcCont = cobra.model.vns.SvcCont(fvTenant, annotation='', userdom='all')
    fvRsTenantMonPol = cobra.model.fv.RsTenantMonPol(fvTenant, annotation='', tnMonEPGPolName='', userdom='all')
    fvEpTags = cobra.model.fv.EpTags(fvTenant, annotation='', userdom='all')


    # commit the generated code to APIC
    print(toXMLStr(polUni))
    c = cobra.mit.request.ConfigRequest()
    c.addMo(polUni)
    md.commit(c)

In [36]:
def createvrf(md,tenantname,vrfname):

    # the top level object on which operations will be made
    polUni = cobra.model.pol.Uni('')
    fvTenant = cobra.model.fv.Tenant(polUni, 'Daniel')

    # build the request using cobra syntax
    fvCtx = cobra.model.fv.Ctx(fvTenant, annotation='', bdEnforcedEnable='no', descr='', ipDataPlaneLearning='enabled', knwMcastAct='permit', name='Daniel', nameAlias='', ownerKey='', ownerTag='', pcEnfDir='ingress', pcEnfPref='enforced', userdom=':all:', vrfIndex='0')
    leakRoutes = cobra.model.leak.Routes(fvCtx, annotation='', descr='', name='', nameAlias='', userdom=':all:')
    fvRsVrfValidationPol = cobra.model.fv.RsVrfValidationPol(fvCtx, annotation='', tnL3extVrfValidationPolName='', userdom='all')
    vzAny = cobra.model.vz.Any(fvCtx, annotation='', descr='', matchT='AtleastOne', name='', nameAlias='', prefGrMemb='disabled', userdom='all')
    vzRsAnyToProv = cobra.model.vz.RsAnyToProv(vzAny, annotation='', intent='install', matchT='AtleastOne', prio='unspecified', tnVzBrCPName='permit-any', userdom=':all:')
    vzRsAnyToCons = cobra.model.vz.RsAnyToCons(vzAny, annotation='', intent='install', prio='unspecified', tnVzBrCPName='permit-any', userdom=':all:')
    fvRsOspfCtxPol = cobra.model.fv.RsOspfCtxPol(fvCtx, annotation='', tnOspfCtxPolName='', userdom='all')
    fvRsCtxToEpRet = cobra.model.fv.RsCtxToEpRet(fvCtx, annotation='', tnFvEpRetPolName='', userdom='all')
    fvRsCtxToExtRouteTagPol = cobra.model.fv.RsCtxToExtRouteTagPol(fvCtx, annotation='', tnL3extRouteTagPolName='', userdom='all')
    fvRsBgpCtxPol = cobra.model.fv.RsBgpCtxPol(fvCtx, annotation='', tnBgpCtxPolName='', userdom='all')


    # commit the generated code to APIC
    print(toXMLStr(fvTenant))
    c = cobra.mit.request.ConfigRequest()
    c.addMo(fvTenant)
    md.commit(c)

In [37]:
def createanycontract(md,tenantname,contractname):




    # the top level object on which operations will be made
    polUni = cobra.model.pol.Uni('')
    fvTenant = cobra.model.fv.Tenant(polUni, tenantname)

    # build the request using cobra syntax
    vzBrCP = cobra.model.vz.BrCP(fvTenant, annotation='', descr='', intent='install', name=contractname, nameAlias='', ownerKey='', ownerTag='', prio='unspecified', targetDscp='unspecified', userdom=':all:')
    vzSubj = cobra.model.vz.Subj(vzBrCP, annotation='', consMatchT='AtleastOne', descr='', name=contractname, nameAlias='', prio='unspecified', provMatchT='AtleastOne', revFltPorts='yes', targetDscp='unspecified', userdom=':all:')
    vzRsSubjFiltAtt = cobra.model.vz.RsSubjFiltAtt(vzSubj, action='permit', annotation='', directives='', priorityOverride='default', tnVzFilterName='default', userdom=':all:')


    # commit the generated code to APIC
    print(toXMLStr(fvTenant))
    c = cobra.mit.request.ConfigRequest()
    c.addMo(fvTenant)
    md.commit(c)

In [38]:
def createapp(md,tenantname,appname):    
    polUni = cobra.model.pol.Uni('')
    fvTenant = cobra.model.fv.Tenant(polUni, tenantname)

    # build the request using cobra syntax
    fvAp = cobra.model.fv.Ap(fvTenant, annotation='', descr='', name=appname, nameAlias='', ownerKey='', ownerTag='', prio='unspecified', userdom=':all:')


    # commit the generated code to APIC
    print(toXMLStr(fvTenant))
    c = cobra.mit.request.ConfigRequest()
    c.addMo(fvTenant)
    md.commit(c)

In [49]:
def createbd(md,tenantname,subnet,bdname,vrf):   
    # the top level object on which operations will be made
    polUni = cobra.model.pol.Uni('')
    fvTenant = cobra.model.fv.Tenant(polUni, tenantname)

    # build the request using cobra syntax
    fvBD = cobra.model.fv.BD(fvTenant, OptimizeWanBandwidth='no', annotation='', arpFlood='yes', descr='', epClear='no', epMoveDetectMode='', hostBasedRouting='no', intersiteBumTrafficAllow='no', intersiteL2Stretch='no', ipLearning='yes', ipv6McastAllow='no', limitIpLearnToSubnets='yes', llAddr='::', mac='', mcastARPDrop='yes', mcastAllow='no', multiDstPktAct='bd-flood', name=bdname, nameAlias='', ownerKey='', ownerTag='', type='regular', unicastRoute='yes', unkMacUcastAct='proxy', unkMcastAct='flood', userdom=':all:', v6unkMcastAct='flood', vmac='not-applicable')
    fvSubnet = cobra.model.fv.Subnet(fvBD, annotation='', ctrl='', descr='', ip=subnet, ipDPLearning='enabled', name='', nameAlias='', preferred='no', userdom=':all:', virtual='no')
    fvRsMldsn = cobra.model.fv.RsMldsn(fvBD, annotation='', tnMldSnoopPolName='', userdom='all')
    fvRsIgmpsn = cobra.model.fv.RsIgmpsn(fvBD, annotation='', tnIgmpSnoopPolName='', userdom='all')
    fvRsCtx = cobra.model.fv.RsCtx(fvBD, annotation='', tnFvCtxName=vrf, userdom='all')
    fvRsBdToEpRet = cobra.model.fv.RsBdToEpRet(fvBD, annotation='', resolveAct='resolve', tnFvEpRetPolName='', userdom='all')
    fvRsBDToNdP = cobra.model.fv.RsBDToNdP(fvBD, annotation='', tnNdIfPolName='', userdom='all')
    
    # commit the generated code to APIC
    print(toXMLStr(fvTenant))
    c = cobra.mit.request.ConfigRequest()
    c.addMo(fvTenant)
    md.commit(c)

In [51]:
def createbdsubnet(md,tenant,bdname,bdsubnet):    
    polUni = cobra.model.pol.Uni('')
    fvTenant = cobra.model.fv.Tenant(polUni, tenant)
    fvBD = cobra.model.fv.BD(fvTenant, bdname)

    # build the request using cobra syntax
    fvSubnet = cobra.model.fv.Subnet(fvBD, annotation='', ctrl='', descr='', ip=bdsubnet, ipDPLearning='enabled', name='', nameAlias='', preferred='no', userdom=':all:', virtual='no')
    # commit the generated code to APIC
    print(toXMLStr(fvBD))
    c = cobra.mit.request.ConfigRequest()
    c.addMo(fvBD)
    md.commit(c)

In [41]:
def createnodeprofile(md,row):
    # the top level object on which operations will be made
    polUni = cobra.model.pol.Uni('')
    infraInfra = cobra.model.infra.Infra(polUni)

    # build the request using cobra syntax
    infraNodeP = cobra.model.infra.NodeP(infraInfra, annotation='', descr='PHCSF1001-101466-AB24 and PHCSF1002-101465-AB24', name=row['LogicalNodeProfile'], nameAlias='', ownerKey='', ownerTag='', userdom=':all:')
    infraRsAccPortP = cobra.model.infra.RsAccPortP(infraNodeP, annotation='', tDn='uni/infra/accportprof-L1001-1002', userdom=':all:')
    infraLeafS = cobra.model.infra.LeafS(infraNodeP, annotation='', descr='', name='L1001-1002', nameAlias='', ownerKey='', ownerTag='', type='range', userdom=':all:')
    infraRsAccNodePGrp = cobra.model.infra.RsAccNodePGrp(infraLeafS, annotation='', tDn='uni/infra/funcprof/accnodepgrp-Switch_Pol_Group', userdom=':all:')
    infraNodeBlk = cobra.model.infra.NodeBlk(infraLeafS, annotation='', descr='', from_='1001', name='75b36de39e9be876', nameAlias='', to_='1002', userdom=':all:')


    # commit the generated code to APIC
    print(toXMLStr(infraInfra))
    c = cobra.mit.request.ConfigRequest()
    c.addMo(infraInfra)
    md.commit(c)

In [42]:
def createepg(md,tenantname,appname,epgname,bdname,domainname):
    # the top level object on which operations will be made
    polUni = cobra.model.pol.Uni('')
    fvTenant = cobra.model.fv.Tenant(polUni, tenantname)
    fvAp = cobra.model.fv.Ap(fvTenant, appname)

    # build the request using cobra syntax
    fvAEPg = cobra.model.fv.AEPg(fvAp, annotation='', descr='', exceptionTag='', floodOnEncap='disabled', fwdCtrl='', hasMcastSource='no', isAttrBasedEPg='no', matchT='AtleastOne', name=epgname, nameAlias='', pcEnfPref='unenforced', prefGrMemb='exclude', prio='level3', shutdown='no', userdom=':all:')
    fvRsDomAtt = cobra.model.fv.RsDomAtt(fvAEPg, annotation='', bindingType='none', classPref='encap', customEpgName='', delimiter='', encap='unknown', encapMode='auto', epgCos='Cos0', epgCosPref='disabled', instrImedcy='lazy', lagPolicyName='', netflowDir='both', netflowPref='disabled', numPorts='0', portAllocation='none', primaryEncap='unknown', primaryEncapInner='unknown', resImedcy='immediate', secondaryEncapInner='unknown', switchingMode='native', tDn='uni/phys-'+domainname, untagged='no', userdom=':all:', vnetOnly='no')
    fvRsCustQosPol = cobra.model.fv.RsCustQosPol(fvAEPg, annotation='', tnQosCustomPolName='', userdom='all')
    fvRsBd = cobra.model.fv.RsBd(fvAEPg, annotation='', tnFvBDName=bdname, userdom='all')


    # commit the generated code to APIC
    print(toXMLStr(fvAp))
    c = cobra.mit.request.ConfigRequest()
    c.addMo(fvAp)
    md.commit(c)

In [43]:
def createdomepg(md,tenantname,appname,bdname,epgname,domname):
    # log into an APIC and create a directory object

    # the top level object on which operations will be made
    polUni = cobra.model.pol.Uni('')
    fvTenant = cobra.model.fv.Tenant(polUni, tenantname)
    fvAp = cobra.model.fv.Ap(fvTenant, appname)
    fvAEPg = cobra.model.fv.AEPg(fvAp, epgname)

    # build the request using cobra syntax
    fvRsDomAtt = cobra.model.fv.RsDomAtt(fvAEPg, annotation='', bindingType='none', classPref='encap', customEpgName='', delimiter='', encap='unknown', encapMode='auto', epgCos='Cos0', epgCosPref='disabled', instrImedcy='lazy', lagPolicyName='', netflowDir='both', netflowPref='disabled', numPorts='0', portAllocation='none', primaryEncap='unknown', primaryEncapInner='unknown', resImedcy='immediate', secondaryEncapInner='unknown', switchingMode='native', tDn='uni/phys-'+domname, untagged='no', userdom=':all:', vnetOnly='no')


    # commit the generated code to APIC
    print(toXMLStr(fvAEPg))
    c = cobra.mit.request.ConfigRequest()
    c.addMo(fvAEPg)
    md.commit(c)

In [44]:
def createintprofile(md,nodeprofile):

        # the top level object on which operations will be made
    # the top level object on which operations will be made
    polUni = cobra.model.pol.Uni('')
    infraInfra = cobra.model.infra.Infra(polUni)

    # build the request using cobra syntax
    infraAccPortP = cobra.model.infra.AccPortP(infraInfra, annotation='', descr='', name=nodeprofile, nameAlias='', ownerKey='', ownerTag='', userdom=':all:')


    # commit the generated code to APIC
    print(toXMLStr(infraInfra))
    c = cobra.mit.request.ConfigRequest()
    c.addMo(infraInfra)
    md.commit(c)

In [45]:
def createaccessport(md,nodeprofile,intpolicy,portname,port):

    #question on fexid also range possible
    # the top level object on which operations will be made
    polUni = cobra.model.pol.Uni('')
    infraInfra = cobra.model.infra.Infra(polUni)
    infraAccPortP = cobra.model.infra.AccPortP(infraInfra, nodeprofile)

    # build the request using cobra syntax
    infraHPortS = cobra.model.infra.HPortS(infraAccPortP, annotation='', descr='', name=portname, nameAlias='', ownerKey='', ownerTag='', type='range', userdom=':all:')
    infraRsAccBaseGrp = cobra.model.infra.RsAccBaseGrp(infraHPortS, annotation='', fexId='101', tDn='uni/infra/funcprof/accbundle-'+intpolicy, userdom=':all:')
    infraPortBlk = cobra.model.infra.PortBlk(infraHPortS, annotation='', descr='', fromCard='1', fromPort=port, name='block2', nameAlias='', toCard='1', toPort=port, userdom=':all:')


    # commit the generated code to APIC
    print(toXMLStr(infraAccPortP))
    c = cobra.mit.request.ConfigRequest()
    c.addMo(infraAccPortP)
    md.commit(c)

In [46]:
def createaccessportselector(md,tenantname,appname,epgname,vlanname,intpolicy,switchprofile):
    vlan='vlan-'+str(filter(str.isdigit, vlanname))
    # the top level object on which operations will be made
    polUni = cobra.model.pol.Uni('')
    fvTenant = cobra.model.fv.Tenant(polUni, tenantname)
    fvAp = cobra.model.fv.Ap(fvTenant, appname)
    fvAEPg = cobra.model.fv.AEPg(fvAp, epgname)

    # build the request using cobra syntax
    fvRsPathAtt = cobra.model.fv.RsPathAtt(fvAEPg, annotation='', descr='', encap=vlan, instrImedcy='immediate', mode='regular', primaryEncap='unknown', tDn='topology/pod-1/protpaths-'+switchprofile+'/pathep-['+intpolicy+']', userdom=':all:')


    # commit the generated code to APIC
    print(toXMLStr(fvAEPg))
    c = cobra.mit.request.ConfigRequest()
    c.addMo(fvAEPg)
    md.commit(c)

In [47]:
def switchfunct(classname,row,md,vlanpool):
    if classname == 'fvnsVlanInstP':
        createVlanPool(md,row['VlanPool'],row['vlan'])
    elif classname== 'physDomP':
        createPhysDom(md,vlanpool,row['PhysicalDomain'])
        attachdomaep(md,row['AEP'],row['PhysicalDomain'])
    elif classname== 'fvTenant':
        print('Create your tenant')
        createTenant(md,row['Tenant'])
    elif classname == 'vzBrCP':
        createanycontract(md,row['Tenant'],row['contract'])
    elif classname=='fvCtx':
        createvrf(md,row['Tenant'],row['VRF'])
    elif classname=='infraNodeP':
        print('Create leaf profile')
    elif classname=='fvAp':
        createapp(md,row['Tenant'],row['ApplicationProfile'])
    elif classname=='fvBD':
        createbd(md,row['Tenant'],row['BDSubnet'],row['BD'],row['VRF'])
    elif classname=='fvSubnet':
        createbdsubnet(md,row['Tenant'],row['BD'],row['BDSubnet'])
    elif classname=='fvAEPg':
        createepg(md,row['Tenant'],row['ApplicationProfile'],row['EPG'],row['BD'])
    elif classname=='fvRsDomAtt':
        createdomepg(md,row['Tenant'],row['ApplicationProfile'],row['BD'],row['EPG'],row['PhysicalDomain'])
    elif classname=='infraAccPortP':
        createintprofile(md,row['LogicalNodeProfile'])
        print('Create leaf interface profile')
    elif classname=='infraHPortS':
        print('Create access port selector')
        createaccessport(md,row['LogicalNodeProfile'],row['InterfacePolicyGroup'],row['portname'],row['port'])
    elif classname=='fvRsPathAtt':
        print('create access port selector')
        createaccessportselector(md,row['Tenant'],row['ApplicationProfile'],row['EPG'],row['BD'],row['InterfacePolicyGroup'],row['SwitchProfile'])
    else:
        pass   

In [52]:

username="user"
passw=getpass.getpass("enter password:")
ls = cobra.mit.session.LoginSession('https://apic', username, passw)
md = cobra.mit.access.MoDirectory(ls)
md.login()
def load_spread():
    with open(r'C:\Users\user\Downloads\aci_variables.csv', newline='',encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            vlanpool ='uni/infra/vlanns-['+row['VlanPool']+']-static' + '/fvnsVlanInstP'
            physdom= 'uni/phys-'+row['PhysicalDomain']+ '/physDomP'
            aep='uni/infra/attentp-'+row['AEP']+'/infraAttEntityP'
            tenant='uni/tn-'+row['Tenant']+'/fvTenant'
            contract='uni/tn-Daniel/brc-'+row['contract']+'/vzBrCP'
            vrf='uni/tn-'+row['Tenant']+'/ctx-'+row['VRF']+'/fvCtx'
            switchprofile='uni/infra/nprof-'+row['LogicalNodeProfile']+'/infraNodeP'
            app='uni/tn-'+row['Tenant']+'/ap-'+row['ApplicationProfile']+'/fvAp'
            bridgepdomain='uni/tn-Daniel/BD-'+row['BD']+'/fvBD'
            bdsubnet='uni/tn-'+row['Tenant']+'/BD-'+row['BD']+'/fvSubnet'
            epg='uni/tn-'+row['Tenant']+'/ap-'+row['ApplicationProfile']+'/epg-'+row['EPG']+'/fvAEPg'
            domatt='uni/tn-'+row['Tenant']+'/ap-'+row['ApplicationProfile']+'/epg-'+row['EPG']+'/rsdomAtt-[uni/phys-'+row['PhysicalDomain']+ ']'+'/fvRsDomAtt'
            intprofile='uni/infra/accportprof-'+row['LogicalNodeProfile']+'/infraAccPortP'
            accessport='uni/infra/accportprof-'+row['LogicalNodeProfile']+'/hports-'+row['portname']+'-typ-range'+'/infraHPortS'
            staticport='uni/tn-'+row['Tenant']+'/ap-'+row['ApplicationProfile']+'/epg-'+row['EPG']+'/rspathAtt-[topology/pod-1/protpaths-'+row['SwitchProfile']+'/pathep-['+row['InterfacePolicyGroup']+']]'+'/fvRsPathAtt'
            l3out='uni/tn-'+row['Tenant']+'/out-'+row['L3OutName']+'/l3extOut'
            config_check_list=[vlanpool,physdom,aep,tenant,vrf,switchprofile,app,bridgepdomain,bdsubnet,domatt,intprofile,staticport,l3out]
            for config in config_check_list:
                x=querydnjson_funct(config)
                if not x:
                    print('Missing: '+str(config))
                    answer=input("Would you like to see all classes relating(y or n):")
                    if answer =='y' or answer =='Y':
                        print('Found related:')
                        temp_list=classQueryFunc(config)
                        for item in temp_list:
                            print(item)
                    elif answer =='n' or answer =='N':
                        pass
                    else:
                        print('Did not recognize your input')
                        
                    answer=input("Would you like to create missing item(y or n):")
                    if answer =='y' or answer =='Y':
                        print('creating '+parseClass(config)+':')
                        switchfunct(parseClass(config),row,md,vlanpool)
                        
                    elif answer =='n' or answer =='N':
                        pass
                    else:
                        print('Did not recognize your input')
                    
                else:
                    print('Found: '+str(config))
load_spread()

enter password: ········


Found: uni/infra/vlanns-[Daniel_Vlan_Pool]-static/fvnsVlanInstP
Found: uni/phys-Daniel_Dom/physDomP
Found: uni/infra/attentp-PITC_AEP/infraAttEntityP
Found: uni/tn-Daniel/fvTenant
Found: uni/tn-Daniel/ctx-Daniel/fvCtx
Found: uni/infra/nprof-L1001-1002/infraNodeP
Found: uni/tn-Daniel/ap-AP/fvAp
Found: uni/tn-Daniel/BD-BD-1161/fvBD
Found: uni/tn-Daniel/BD-BD-1161/fvSubnet
Found: uni/tn-Daniel/ap-AP/epg-EPG-1161/rsdomAtt-[uni/phys-Daniel_Dom]/fvRsDomAtt
Found: uni/infra/accportprof-L1001-1002/infraAccPortP
Found: uni/tn-Daniel/ap-AP/epg-EPG-1161/rspathAtt-[topology/pod-1/protpaths-1001-1002/pathep-[Test_PO1]]/fvRsPathAtt
Found: uni/tn-Daniel/out-L3-Out-Daniel/l3extOut
Found: uni/infra/vlanns-[Tim_Vlan]-static/fvnsVlanInstP
Found: uni/phys-Tim_Dom/physDomP
Found: uni/infra/attentp-PITC_AEP/infraAttEntityP
Found: uni/tn-Tim/fvTenant
Missing: uni/tn-Tim/ctx-Daniel/fvCtx


Would you like to see all classes relating(y or n): n
Would you like to create missing item(y or n): y


creating fvCtx:
<?xml version="1.0" encoding="UTF-8"?>
<fvTenant name='Daniel' status='created,modified'><fvCtx annotation='' bdEnforcedEnable='no' descr='' ipDataPlaneLearning='enabled' knwMcastAct='permit' name='Daniel' nameAlias='' ownerKey='' ownerTag='' pcEnfDir='ingress' pcEnfPref='enforced' status='created,modified' userdom=':all:' vrfIndex='0'><leakRoutes annotation='' descr='' name='' nameAlias='' status='created,modified' userdom=':all:'></leakRoutes><fvRsVrfValidationPol annotation='' status='created,modified' tnL3extVrfValidationPolName='' userdom='all'></fvRsVrfValidationPol><vzAny annotation='' descr='' matchT='AtleastOne' name='' nameAlias='' prefGrMemb='disabled' status='created,modified' userdom='all'><vzRsAnyToProv annotation='' intent='install' matchT='AtleastOne' prio='unspecified' status='created,modified' tnVzBrCPName='permit-any' userdom=':all:'></vzRsAnyToProv><vzRsAnyToCons annotation='' intent='install' prio='unspecified' status='created,modified' tnVzBrCPName

Would you like to see all classes relating(y or n): n
Would you like to create missing item(y or n): n


...

Would you like to see all classes relating(y or n): y


Found related:
uni/tn-Stan_The_Man/ap-AP/epg-EPG-1081/rspathAtt-[topology/pod-1/protpaths-1001-1002/pathep-[Test_PO1]]
uni/tn-Mirwais/ap-AP/epg-EPG-1031/rspathAtt-[topology/pod-1/protpaths-1001-1002/pathep-[Test_PO1]]
uni/tn-Test/ap-AP/epg-EPG-1004/rspathAtt-[topology/pod-1/protpaths-1017-1018/pathep-[Test_PO2]]
uni/tn-Austin-N/ap-Austin-AP/epg-EPG-1011/rspathAtt-[topology/pod-1/protpaths-1001-1002/pathep-[Test_PO1]]
uni/tn-Steve/ap-AP/epg-EPG-1072/rspathAtt-[topology/pod-1/protpaths-1017-1018/pathep-[Test_PO2]]
uni/tn-Mirwais/ap-AP/epg-EPG-1032/rspathAtt-[topology/pod-1/protpaths-1017-1018/pathep-[Test_PO2]]
uni/tn-Austin-N/ap-Austin-AP/epg-EPG-1012/rspathAtt-[topology/pod-1/protpaths-1017-1018/pathep-[Test_PO2]]
uni/tn-Test/ap-AP/epg-EPG-1001/rspathAtt-[topology/pod-1/paths-1001/pathep-[eth1/15]]
uni/tn-Stan_The_Man/ap-AP/epg-EPG-1082/rspathAtt-[topology/pod-1/protpaths-1017-1018/pathep-[Test_PO2]]
uni/tn-Test/ap-AP/epg-EPG-1002/rspathAtt-[topology/pod-1/protpaths-1001-1002/pathep-[T

Would you like to create missing item(y or n): n


Missing: uni/tn-Tim/out-L3-tem/l3extOut


Would you like to see all classes relating(y or n): n
Would you like to create missing item(y or n): n


### Print tenant bd in json format...you need the config to print it obviously...config not listed here

In [ ]:
import json
from cobra.internal.codec.jsoncodec import toJSONStr
for tenantBd in tenantBdMoList:
    x=(json.loads(toJSONStr(tenantBd)))
    print(json.dumps(x,sort_keys=True, indent=4))

### query function that get's passed a class

In [20]:
def query_class(temp_class):
    class_query = cobra.mit.access.ClassQuery(temp_class)
    objs = md.query(class_query)
    for item in objs:
        print(item.name)

### I take a config and break out subsets

In [194]:
 if(x["fvTenant"]["attributes"]["name"])=="Daniel":
        print('found tenant')
        temp_list=(x["fvTenant"]["children"])
        for lst in temp_list:
            if type(lst)==dict:
                if "fvBD" in lst:
                    print(lst["fvBD"]["attributes"]["name"])
                    fvBDChildren=(lst["fvBD"]["children"])
                    for item in fvBDChildren:
                        if type(item)==dict:
                            if "fvSubnet" in item:
                                print("   "+item["fvSubnet"]["attributes"]["ip"])
                if "vzBrCP" in lst:
                    print('contract:'+lst["vzBrCP"]["attributes"]["name"])
                if "fvCtx" in lst:
                    print('VRF:'+lst["fvCtx"]["attributes"]["name"])
                if "fvAp" in lst:
                    print(lst["fvAp"]["attributes"]["name"])
                    fvAp=(lst["fvAp"]["children"])
                    for itemz in fvAp:
                        if type(itemz)==dict:
                            if "fvAEPg" in itemz:
                                fvAEPg=(itemz["fvAEPg"]["children"])
                                for itmz in fvAEPg:
                                    if "fvRsDomAtt" in itmz:
                                        print("   "+itemz["fvAEPg"]["attributes"]["name"]) 
                                        print("      "+itmz["fvRsDomAtt"]["attributes"]["tDn"])
                                
                                

found tenant
contract:permit-any
VRF:Daniel
BD-1162
   10.17.2.1/24
BD-1161
   10.17.1.1/24
AP
   EPG-1162
      uni/phys-Daniel_Dom
   EPG-1161
      uni/phys-Daniel_Dom
